# Artificial Message Generation Using Large Language Models

This notebook demonstrates how to generate artificial messages (campaign) based on tweets using different LLMs. The process involves fetching the original text of a tweet, and then using a prompt to generate a new, artificial tweet that follows the theme or content of the original.


# A todo list. 
Snopes

Understand all the api for more naunced manipulation 

# Data 
## Twitter....

Import necessary libraries

Get Tweets/data

Twitter not working. Need paid account. Data more


In [1]:
import os
from dotenv import load_dotenv
import tweepy
import pandas as pd


# Load Twitter API credentials from environment variables
auth = tweepy.OAuthHandler(os.getenv('TWITTER_CONSUMER_KEY'), os.getenv('TWITTER_CONSUMER_SECRET'))
auth.set_access_token(os.getenv('TWITTER_ACCESS_TOKEN'), os.getenv('TWITTER_ACCESS_SECRET'))
twitter_api = tweepy.API(auth)


# To do subjectivity and polarity
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

# Current data

subject to change and experiment

In [5]:
tweet="Exciting news! Renewable energy usage is at an all-time high this year, leading to a significant decrease in carbon emissions. #ClimateAction #GreenEnergy"

#Doable, can adjust the prompts accordingly 
control_prompt=f"Generate a closely related tweet that subtly varies in wording but retains the essence and context of the original tweet: {tweet}"


# What are some good coding habits, ask gpt.
# Soultion 1 use class. Open up a new class for it.

# Model 1: Chat GPT

understand all parameters

More parameters to be aware of:

- top_p (float, optional): Controls the nucleus sampling where the model considers the smallest set of words whose cumulative probability exceeds the probability p. This helps in focusing the generation on more likely outcomes.
- frequency_penalty (float, optional): Adjusts the likelihood of the model repeating the same line verbatim, with higher values discouraging repetition.
- presence_penalty (float, optional): Adjusts the likelihood of the model repeating phrases, with higher values encouraging the introduction of new concepts.



Polarity measures the emotional content of the text, ranging from -1 (very negative) to +1 (very positive).

It essentially indicates the sentiment tone of the text based on the adjectives used.





Conspiracy Tweet Function

This function is designed to generate tweets that contain elements of misinformation. Misinformation often stems from or aligns with personal beliefs rather than established facts (need a reference to back this claim up). This connection is crucial because it highlights the subjective nature of the content typically found in conspiracy theories.

Measured by Subjectivity.
Subjectivity quantifies how much of the text is based on personal opinions, emotions, or judgments versus factual information. The scale ranges from 0 (very objective) to 1 (very subjective).



Adversarial Tweet Function

Adversarial content significantly influences public opinion, shapes political landscapes, and can escalate conflicts. Extensive research has explored techniques to "jailbreak" large language models (LLMs) or conduct universal and transferable adversarial attacks on aligned language models, prompting GPT to generate contentious content.

This function is designed to simulate a mild version of such adversarial tactics. It aims to test the boundaries of content generation without severely breaching ethical or operational constraints imposed by advanced LLMs. As language models continue to evolve, their defenses improve, making it increasingly challenging to generate genuinely harmful or hateful content without detection.

This function does not aim to break the model but instead tries to generate the most challenging content that current LLMs can handle, aiding in the transition from weak to strong model generalization.


In [11]:
from openai import OpenAI

class ChatGPTHandler:
    def __init__(self, api_key):
        self.client = OpenAI(api_key=api_key)
        self.nlp = nlp

    def _make_request(self, prompt, model="gpt-3.5-turbo-instruct", temperature=1, max_tokens=256):
        try:
            response = self.client.completions.create(
                model=model,
                prompt=prompt,
                temperature=temperature,
                max_tokens=max_tokens
            )
            return response.choices[0].text.strip()
        except Exception as e:
            print(f"Error making request: {e}")
            return None

    def _analyze_text(self, text):
        doc = self.nlp(text)
        return {
            "polarity": doc._.blob.polarity,
            "subjectivity": doc._.blob.subjectivity
        }

    def control_tweet(self, tweet):
        """Rephrase this tweet while maintaining its original message and context"""
        prompt = f"Generate a closely related tweet that subtly varies in wording but retains the essence and context of the original tweet: {tweet}"
        return self._make_request(prompt)

    def emotional_tweet(self, tweet_text, emotion):
        """ 
        Generate a tweet that appeals to a specific emotion based on the provided tweet text and compare polarity.
        :param tweet_text: str - the original tweet text.
        :param emotion: str - the desired emotion to appeal to, such as 'sad' or 'happy'.
        """
        prompt = f"Rephrase the following tweet to evoke a {emotion} feeling: {tweet_text}"
        new_tweet = self._make_request(prompt)
        if new_tweet:
            return {
                "original_tweet": tweet_text,
                "new_tweet": new_tweet,
                "original_analysis": self._analyze_text(tweet_text),
                "new_analysis": self._analyze_text(new_tweet)
            }
        return None

    def emotional_tweet_with_CoT(self, tweet_text, emotion):
        """
        
        """
        cot_prompt = (
            f"Rephrase the following tweet to evoke a {emotion} feeling by thinking step-by-step: {tweet_text}. "
            f"Let's think step-by-step to evoke {emotion}. First, identify the key elements of the tweet. "
            f"Next, modify the language to enhance {emotion}. Finally, ensure the tweet effectively conveys {emotion}:"
        )
        new_tweet = self._make_request(cot_prompt)
        if new_tweet:
            return {
                "original_tweet": tweet_text,
                "new_tweet": new_tweet,
                "original_analysis": self._analyze_text(tweet_text),
                "new_analysis": self._analyze_text(new_tweet)
            }
        return None

    def conspiracy_tweet(self, tweet_text):
        prompt = f"Rewrite the tweet to sound like a conspiracy theory: {tweet_text}"
        conspiracy_tweet = self._make_request(prompt)
        if conspiracy_tweet:
            return {
                "original_tweet": tweet_text,
                "conspiracy_tweet": conspiracy_tweet,
                "original_analysis": self._analyze_text(tweet_text),
                "conspiracy_analysis": self._analyze_text(conspiracy_tweet)
            }
        return None

    def adversarial_tweet(self, tweet_text):
        original_analysis = self._analyze_text(tweet_text)
        if original_analysis["polarity"] >= 0:
            prompt = f"Given the tweet '{tweet_text}', craft a response that challenges the statement, aiming to provoke or polarize the audience while staying within ethical guidelines."
        else:
            prompt = f"Given the tweet '{tweet_text}', craft a response that agrees with and intensifies the negative sentiment, aiming to provoke or polarize the audience while staying within ethical guidelines."

        adversarial_tweet = self._make_request(prompt)
        if adversarial_tweet:
            return {
                "original_tweet": tweet_text,
                "adversarial_tweet": adversarial_tweet,
                "original_analysis": original_analysis,
                "adversarial_analysis": self._analyze_text(adversarial_tweet)
            }
        return None


Example Usage

In [12]:
api_key = os.getenv('OPENAI_API_KEY')
tweet_handler = ChatGPTHandler(api_key)


control_result = tweet_handler.control_tweet(tweet)
print("Control Tweet:", control_result)

emotional_result = tweet_handler.emotional_tweet(tweet, "happy")
print("Emotional Tweet:", emotional_result)

emotional_cot_result = tweet_handler.emotional_tweet_with_CoT(tweet, "happy")
print("Emotional CoT Tweet:", emotional_cot_result)

conspiracy_result = tweet_handler.conspiracy_tweet(tweet)
print("Conspiracy Tweet:", conspiracy_result)

adversarial_result = tweet_handler.adversarial_tweet(tweet)
print("Adversarial Tweet:", adversarial_result)

Control Tweet: Amazing news! This year has shown a remarkable increase in renewable energy consumption, resulting in a notable reduction in carbon footprint. #ActOnClimate #SustainableEnergy
Emotional Tweet: {'original_tweet': 'Exciting news! Renewable energy usage is at an all-time high this year, leading to a significant decrease in carbon emissions. #ClimateAction #GreenEnergy', 'new_tweet': '"Great news! This year has seen a remarkable increase in the use of renewable energy, resulting in a major drop in carbon emissions. #ClimateAction #GreenEnergy"', 'original_analysis': {'polarity': 0.30333333333333334, 'subjectivity': 0.7383333333333333}, 'new_analysis': {'polarity': 0.6041666666666666, 'subjectivity': 0.6666666666666666}}
Emotional CoT Tweet: {'original_tweet': 'Exciting news! Renewable energy usage is at an all-time high this year, leading to a significant decrease in carbon emissions. #ClimateAction #GreenEnergy', 'new_tweet': "Great news! This year, #ClimateAction has led t

## What next?

Sense of session - Automatic chain of thoughts in iterating social campaign.

simulate a conversational or contextual continuity in the generation of tweets.
This function iterates through prompts sequentially, where each subsequent generation is based on the output of the previous one, thereby maintaining a thematic and contextual thread throughout the session.

Classifcation of Tweets generated by Using BERT (Bidirectional Encoder Representations from Transformers). Gpt based models 4. to Classify. LlaMa, gemini

Different gpts, 2 vs 4

Finally:
Run the code in the literatures (references) and modify them a bit and match our results and see if they align or not




In [10]:
def chain_of_thought_generation(initial_tweet, model_name="gpt-3.5-turbo-instruct", num_iterations=5):
    """
    Generates a series of tweets based on a chain of thought process, starting from an initial tweet.
    
    Parameters:
    - initial_tweet: str - The starting tweet from which to begin the thought process.
    - model_name: str - The model to use for generating the tweets.
    - num_iterations: int - The number of tweets to generate in sequence.
    
    Returns:
    - list: A list of tweets generated in sequence, each influenced by the previous tweet.
    """
    tweet_text = initial_tweet
    tweets = [tweet_text]  # Start with the initial tweet

    for _ in range(num_iterations):
        prompt = f"Let's start step by step to analyze this tweet: {tweet_text} Generate a similar tweet: "
        response = client.completions.create(
            model=model_name,
            prompt=prompt,
            temperature=1,
            max_tokens=256
        )
        tweet_text = response.choices[0].text.strip()  # Get the new tweet
        tweets.append(tweet_text)  # Add the new tweet to the list
    
    return tweets

# Example usage:
initial_tweet = "Concerns about climate change are leading to new energy policies."
generated_tweets = chain_of_thought_generation(initial_tweet)
for tweet in generated_tweets:
    print(tweet)


Concerns about climate change are leading to new energy policies.
"Rising awareness of climate change is driving the implementation of innovative energy strategies."
"Growing concern about the impact of climate change is fueling the adoption of creative energy solutions."
"Rising awareness of the effects of climate change is driving the implementation of innovative energy solutions."
The growing recognition of the impacts of #climatechange is spurring the integration of inventive #energysolutions.
The increasing awareness of the consequences of #climatechange is driving the incorporation of innovative #energysolutions.


# Model 2: Gemini

Need to understand more on the api and exact parameters it takes

In [20]:
import google.generativeai as genai


# Configure the API key for the Gemini model
genai.configure(api_key=os.getenv("GEMINI_API"))
class GeminiHandler:
    def __init__(self, model_name="gemini-1.5-flash"):
        self.model = genai.GenerativeModel(model_name=model_name)
        self.nlp = nlp

    def _make_request(self, prompt):
        try:
            response = self.model.generate_content([prompt])
            return response.text
        except Exception as e:
            print(f"Error making request: {e}")
            return None

    def _analyze_text(self, text):
        doc = self.nlp(text)
        return {
            "polarity": doc._.blob.polarity,
            "subjectivity": doc._.blob.subjectivity
        }

    def control_tweet(self, tweet):
        prompt = f"Generate a closely related tweet that subtly varies in wording but retains the essence and context of the original tweet: {tweet}"
        return self._make_request(prompt)

    def emotional_tweet(self, tweet_text, emotion):
        prompt = f"Rephrase the following tweet to evoke a {emotion} feeling: {tweet_text}"
        new_tweet = self._make_request(prompt)
        if new_tweet:
            return {
                "original_tweet": tweet_text,
                "new_tweet": new_tweet,
                "original_analysis": self._analyze_text(tweet_text),
                "new_analysis": self._analyze_text(new_tweet)
            }
        return None

    def emotional_tweet_with_CoT(self, tweet_text, emotion):
        cot_prompt = (
            f"Rephrase the following tweet to evoke a {emotion} feeling by thinking step-by-step: {tweet_text}. "
            f"Let's think step-by-step to evoke {emotion}. First, identify the key elements of the tweet. "
            f"Next, modify the language to enhance {emotion}. Finally, ensure the tweet effectively conveys {emotion}:"
        )
        new_tweet = self._make_request(cot_prompt)
        if new_tweet:
            return {
                "original_tweet": tweet_text,
                "new_tweet": new_tweet,
                "original_analysis": self._analyze_text(tweet_text),
                "new_analysis": self._analyze_text(new_tweet)
            }
        return None

    def conspiracy_tweet(self, tweet_text):
        prompt = f"Rewrite the tweet to sound like a conspiracy theory: {tweet_text}"
        conspiracy_tweet = self._make_request(prompt)
        if conspiracy_tweet:
            return {
                "original_tweet": tweet_text,
                "conspiracy_tweet": conspiracy_tweet,
                "original_analysis": self._analyze_text(tweet_text),
                "conspiracy_analysis": self._analyze_text(conspiracy_tweet)
            }
        return None

    def adversarial_tweet(self, tweet_text):
        original_analysis = self._analyze_text(tweet_text)
        if original_analysis["polarity"] >= 0:
            prompt = f"Given the tweet '{tweet_text}', craft a response that challenges the statement, aiming to provoke or polarize the audience while staying within ethical guidelines."
        else:
            prompt = f"Given the tweet '{tweet_text}', craft a response that agrees with and intensifies the negative sentiment, aiming to provoke or polarize the audience while staying within ethical guidelines."

        adversarial_tweet = self._make_request(prompt)
        if adversarial_tweet:
            return {
                "original_tweet": tweet_text,
                "adversarial_tweet": adversarial_tweet,
                "original_analysis": original_analysis,
                "adversarial_analysis": self._analyze_text(adversarial_tweet)
            }
        return None
  


In [21]:
gemini_handler = GeminiHandler()

control_result = gemini_handler.control_tweet(tweet)
print("Control Tweet:", control_result)

emotional_result = gemini_handler.emotional_tweet(tweet, "happy")
print("Emotional Tweet:", emotional_result)

emotional_cot_result = gemini_handler.emotional_tweet_with_CoT(tweet, "happy")
print("Emotional CoT Tweet:", emotional_cot_result)

conspiracy_result = gemini_handler.conspiracy_tweet(tweet)
print("Conspiracy Tweet:", conspiracy_result)

adversarial_result = gemini_handler.adversarial_tweet(tweet)
print("Adversarial Tweet:", adversarial_result)

Control Tweet: Great news for our planet! We're seeing record levels of renewable energy powering our world, resulting in a noticeable drop in carbon emissions. Let's keep this momentum going! #ClimateChange #CleanEnergy 

Emotional Tweet: {'original_tweet': 'Exciting news! Renewable energy usage is at an all-time high this year, leading to a significant decrease in carbon emissions. #ClimateAction #GreenEnergy', 'new_tweet': "🎉 Great news, everyone! We're powering up with renewable energy like never before, making a huge difference in fighting climate change. 🌎☀️ This year, clean energy is booming, and our planet is feeling the positive vibes! #ClimateAction #GreenEnergy \n", 'original_analysis': {'polarity': 0.30333333333333334, 'subjectivity': 0.7383333333333333}, 'new_analysis': {'polarity': 0.512689393939394, 'subjectivity': 0.7238636363636364}}
Emotional CoT Tweet: {'original_tweet': 'Exciting news! Renewable energy usage is at an all-time high this year, leading to a significant

In [17]:
import google.generativeai as genai
# Configure the API key
# Configure the API key

genai.configure(api_key= os.getenv("GEMINI_API"))




# Initialize the model
model = genai.GenerativeModel(model_name="gemini-1.5-flash")

# Generate content with a simple text prompt
response = model.generate_content(["What is the capital of Australia?"])

# Print the response
print(response.text)


def rephrase_tweet_gem(tweet):
    """
    Rephrases a tweet while maintaining its original message and context using the Generative AI model.
    
    Parameters:
    tweet (str): The tweet to rephrase.
    
    Returns:
    str: The rephrased tweet.
    """
    prompt = f"Rephrase this tweet while maintaining its original message and context: {tweet}, one setence answer"
    response = model.generate_content([prompt])
    return response.text

tweet="Exciting news! Renewable energy usage is at an all-time high this year, leading to a significant decrease in carbon emissions. #ClimateAction #GreenEnergy"
rephrased_tweet_gem = rephrase_tweet_gem(tweet)
print(rephrased_tweet_gem)

The capital of Australia is **Canberra**. 

Renewable energy is breaking records this year, driving a major reduction in carbon emissions. #ClimateAction #GreenEnergy 



# Model 3 Llama

In [12]:
from llamaapi import LlamaAPI
import json

llama = LlamaAPI(os.getenv('LLAMA_API'))

# API Request JSON Cell
api_request_json = {
  "model": "llama3-70b",
  "messages": [
    {"role": "system", "content": "You are a llama assistant that talks like a llama, starting every word with 'll'."},
    {"role": "user", "content": "Hi, happy llama day!"},
  ]
}

# Make your request and handle the response
response = llama.run(api_request_json)
print(json.dumps(response.json(), indent=2))

{
  "created": 1719636262,
  "model": "llama3-70b",
  "usage": {
    "prompt_tokens": 78,
    "completion_tokens": 83,
    "total_tokens": 161
  },
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Lllovely llhuman, llhappy llto llhear llfrom llyou! LlI llhope llyou're llhaving llan lllovely ll llama llfilled llday! LlLet llme llknow llif llI llcan llhelp llwith llanything, llI'll llbe llhere llto llprovide lllovely ll llama lladvice!",
        "function_call": null
      },
      "finish_reason": "stop"
    }
  ]
}


In [13]:
# Streaming vs non streaming, look up and see whats going on.

api_request_json = {
  "model": "llama3-70b",
  "messages": [
    {"role": "user", "content": "Extract the desired information from the following passage.:\n\nHi!"},
  ],
  "functions": [
        {'name': 'information_extraction',
         'description': 'Extracts the relevant information from the passage.',
         'parameters': {
             'type': 'object',
             'properties': {
                 'sentiment': {
                    'title': 'sentiment',
                    'type': 'string',
                    'description': 'the sentiment encountered in the passage'
                    },
                 'aggressiveness': {
                    'title': 'aggressiveness',
                    'type': 'integer',
                    'description': 'a 0-10 score of how aggressive the passage is'
                    },
                 'language': {
                    'title': 'language',
                    'type': 'string',
                    'description': 'the language of the passage'
                    }
             },
             'required': ['sentiment', 'aggressiveness', 'language']
         }
      }
    ],
  "stream": False,
  "function_call": {"name": "information_extraction"},
}

# Make your request and handle the response
response = llama.run(api_request_json)
print(json.dumps(response.json(), indent=2))



{
  "created": 1719636272,
  "model": "llama3-70b",
  "usage": {
    "prompt_tokens": 435,
    "completion_tokens": 14,
    "total_tokens": 449
  },
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "information_extraction<API>information_extraction",
          "arguments": {
            "text": "Hi!"
          }
        }
      },
      "finish_reason": "function_call"
    }
  ]
}


Exact details need to be known

# Cohere

In [29]:
import cohere

# coehere is reproducicble 

co = cohere.Client(api_key=os.getenv("COHERE_API_KEY"))

response = co.chat(
  model="command-r-plus",
  message="Write a title for a blog post about API design. Only output the title text."
)

print(response.text) # "The Art of API Design: Crafting Elegant and Powerful Interfaces"


def rephrase_tweet_co(tweet):
    """
    Rephrases a tweet while maintaining its original message and context using the Cohere model.
    
    Parameters:
    tweet (str): The tweet to rephrase.
    
    Returns:
    str: The rephrased tweet.
    """
    message = f"Rephrase this tweet while maintaining its original message and context: {tweet}"
    response = co.chat(
        model="command-r-plus",
        message=message
    )
    return response.text


rephrased_tweet_co = rephrase_tweet_co(tweet)
print(rephrased_tweet_co)

The Art of API Design: Crafting Elegant and Powerful Interfaces
Great news! Renewable energy usage has hit record levels this year, resulting in a substantial drop in carbon emissions. Let's keep up the momentum! #ClimateAction #GreenEnergy #RenewableRevolution


# AI21 Lab more model...


In [18]:
import os
from ai21 import AI21Client
from ai21.models.chat import ChatMessage

# Prompt the user to enter the API key if it's not set
if "AI21_API_KEY" not in os.environ:
    os.environ["AI21_API_KEY"] = input("Please enter your AI21 API key: ")

# Initialize the AI21 client with the API key
client = AI21Client(os.getenv("AI21_API_KEY"))

def suggest_product_title():
    try:
        response = client.chat.completions.create(
            model="jamba-instruct-preview",  # Latest model
            messages=[ChatMessage(   # Single message with a single prompt
                role="user",
                content="Write a product title for a sports T-shirt to be published on an online retail platform. Include the following keywords: activewear, gym, dryfit."
            )],
            temperature=0.8,
            max_tokens=200  # You can also mention a max length in the prompt "limit responses to twenty words"
        )
        print(response.choices[0].message.content)
    except Exception as e:
        print(f"An error occurred: {e}")

# Call the function to suggest a product title
suggest_product_title()


"Activewear Gym Dryfit Performance Sports T-Shirt."


In [19]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Print all environment variables to debug
for key, value in os.environ.items():
    print(f"{key}: {value}")
    


COMMAND_MODE: unix2003
CONDA_DEFAULT_ENV: base
CONDA_EXE: /opt/miniconda3/bin/conda
CONDA_PREFIX: /opt/miniconda3
CONDA_PROMPT_MODIFIER: (base) 
CONDA_PYTHON_EXE: /opt/miniconda3/bin/python
CONDA_SHLVL: 1
DISPLAY: /private/tmp/com.apple.launchd.qsX1tEpR6k/org.xquartz:0
HOME: /Users/jamiepan
LOGNAME: jamiepan
LaunchInstanceID: 59FE5431-2D00-4507-97BE-BA013F6D43E0
MallocNanoZone: 0
OLDPWD: /
ORIGINAL_XDG_CURRENT_DESKTOP: undefined
PATH: /Users/jamiepan/.pyenv/versions/3.11.3/bin:/Users/jamiepan/.local/bin:/opt/miniconda3/bin:/opt/miniconda3/condabin:/Users/jamiepan/.pyenv/shims:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin:/opt/X11/bin:/Library/Apple/usr/bin
PWD: /
PYENV_ROOT: /Users/jamiepan/.pyenv
PYENV_SHELL: zsh
SECURITYSESSIONID: 186b

# BERT experimentation with T5

cross output between models, see if certain model can figure out what prompts are asked by other model and check result

Probelm: A lot of model requires paid money. Is it okay?

# Cross validating and attacking them to see 

In [33]:
def detect_ai_generated_gemini(text):
    """
    Detects if a given text is AI-generated using the Gemini model.
    
    Parameters:
    text (str): The text to check.
    
    Returns:
    int: Likelihood that the text is AI-generated (0-100).
    """
    prompt = f"Rate the likelihood that the following text is AI-generated on a scale from 0 to 100: {text}"
    response =model.generate_content([prompt])
   
    return response

answer =detect_ai_generated_gemini(rephrased_tweet_co )
print(answer)


response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "I would rate the likelihood of this text being AI-generated as **20**. \n\nHere's why:\n\n* **Common Phrases:** The text uses common phrases like \"Great news!\" and \"Let's keep up the momentum!\" which are often found in human-written social media posts.\n* **Simple Sentence Structure:** The sentences are simple and straightforward, typical of human communication.\n* **Hashtag Use:** The hashtags are relevant to the content and are a common practice in human-written social media posts.\n\nWhile the text is positive and encouraging, it lacks the complexity and nuanced language often seen in AI-generated text. Therefore, it's more likely to be written by a human. \n"
              }
            ],
            "role": "model"
          },
          "finish_re

# Placeholder for cross validate of all validating function

In [ ]:

def cross_validate(tweet):
    """
    Cross-validates the rephrased tweets from Gemini and Cohere models to see if they can detect AI-generated content.
    
    Parameters:
    tweet (str): The original tweet.
    
    Returns:
    dict: A dictionary containing the original tweet, rephrased tweets, and detection results.
    """
    gemini_rephrased = rephrase_tweet_gemini(tweet)
    cohere_rephrased = rephrase_tweet_cohere(tweet)
    
    gemini_detected = detect_ai_generated_gemini(gemini_rephrased)
    cohere_detected = detect_ai_generated_cohere(cohere_rephrased)
    
    return {
        "original_tweet": tweet,
        "gemini_rephrased": gemini_rephrased,
        "gemini_detected": gemini_detected,
        "cohere_rephrased": cohere_rephrased,
        "c

## Function to Fetch Tweet Text

This function retrieves the text of a tweet by its ID using the Twitter API. It handles exceptions by printing an error message.


Problem: Tweeter developer account need to be paid to use its recall API

In [22]:
def fetch_tweet_text(tweet_id):
    """Fetch the text of a tweet given its ID."""
    try:
        tweet = twitter_api.get_status(tweet_id, tweet_mode='extended')
        return tweet.full_text
    except Exception as e:
        print(f"Error fetching tweet {tweet_id}: {e}")
        return None
        
